### Observability

We use Arize Phoenix for observability. You can access tje Phoenix app in your browser by visiting `http://localhost:6006/` after running the following code.

In [3]:
# setup Phoenix
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


Load the environment variables and the dataset

In [8]:
import os
from dotenv import load_dotenv

import pandas as pd

load_dotenv()

df = pd.read_csv("./data/ags.csv")

In [9]:
df.shape

(919, 18)

In [11]:
df.head()

,LOTID,CREATIONDATE,MODIFICATIONDATE,MISCOTHERCOST,FEEDCOST,PROCESSINGCOST,VETCOST,RAILERWT,RAILERHEADCOUNT,RAILERSALES,CATTLECOST,TOTALPAYWTIN,TOTALOFFTRUCKWT,SEXTYPELOT,HEADINLOT,FEEDYARDPHONE,STARTPERIOD,ENDPERIOD
0,S209044,29/07/2020,10/09/2021,2713.70,126065.57,2341.11,1298.60,1830,3,333.00,40164.17,36740,36780,X,117,509-547-8864,29/07/2020,00:00.0
1,S209047,09/08/2020,10/09/2021,3212.39,252330.47,4540.65,983.61,1400,2,626.54,122918.81,82405,80470,X,225,509-547-8864,09/08/2020,00:00.0
2,S207052,21/06/2020,27/01/2021,414.63,48101.11,849.93,480.46,1150,1,276.25,0.01,94492,91670,NaN,100,509-547-8864,21/06/2020,00:00.0
3,S209040,01/07/2020,07/06/2021,2931.72,173130.17,2043.68,1106.20,3020,3,756.25,145013.51,154977,152720,X,210,509-547-8864,01/07/2020,00:00.0
4,S209049,11/08/2020,30/09/2021,2343.84,192479.95,2595.63,1017.13,1010,1,245.00,61832.47,58066,56020,X,165,509-547-8864,11/08/2020,00:00.0


## Format dates

We want to format all columns with dates so that .dt can be used to access the date components.

In [12]:
df['CREATIONDATE'] = pd.to_datetime(df['CREATIONDATE'], format="mixed")
df['MODIFICATIONDATE'] = pd.to_datetime(df['MODIFICATIONDATE'], format="mixed")
df['STARTPERIOD'] = pd.to_datetime(df['STARTPERIOD'], format="mixed")
df['ENDPERIOD'] = pd.to_datetime(df['ENDPERIOD'], format="mixed")


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   LOTID             919 non-null    object        
 1   CREATIONDATE      919 non-null    datetime64[ns]
 2   MODIFICATIONDATE  919 non-null    datetime64[ns]
 3   MISCOTHERCOST     919 non-null    float64       
 4   FEEDCOST          919 non-null    float64       
 5   PROCESSINGCOST    919 non-null    float64       
 6   VETCOST           919 non-null    float64       
 7   RAILERWT          919 non-null    int64         
 8   RAILERHEADCOUNT   919 non-null    int64         
 9   RAILERSALES       919 non-null    float64       
 10  CATTLECOST        919 non-null    float64       
 11  TOTALPAYWTIN      919 non-null    int64         
 12  TOTALOFFTRUCKWT   919 non-null    int64         
 13  SEXTYPELOT        753 non-null    object        
 14  HEADINLOT         919 non-

In [23]:
from llama_index.llms.together import TogetherLLM
from llama_index.core.settings import Settings
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import PandasInstructionParser

In [40]:
llm = TogetherLLM(
    model="meta-llama/Llama-3-70b-chat-hf", api_key=os.environ['TOGETHER_API_KEY']
)


Settings.llm = llm

We provide the model with the context of what each column represents.

In [27]:
df_info_str = """
LOTID: Unique key for each listing, same value as the 'key' used in the listing object in the API.
CREATIONDATE: The date the listing was created.
MODIFICATIONDATE: The miscother cost.
MISCOTHERCOST: The Credit rating that was assigned at the time the listing went live. Applicable for listings pre-2009 period and will only be populated for those listings.
FEEDCOST: The cost of the feed.
PROCESSINGCOST: The processing cost.
VETCOST: Cost of the vet.
RAILERWT: Railer wt.
RAILERHEADCOUNT: Total headcounts of railers.
RAILERSALES: The sales of the railer.
CATTLECOST: The cost of the cattle.
TOTALPAYWTIN: Total payment of wt.
TOTALOFFTRUCKWT:  Total off truck wt.
SEXTYPELOT (numeric): Sex type of the lot.
HEADINLOT (Alpha): Total heaads in lot.
FEEDYARDPHONE: Phone number of feed yard.
STARTPERIOD: The date the listing period was started.
ENDPERIOD: The date the listing period was ended.
"""

In [28]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. GIVE ONLY THE EXPRESSION AND NOTHING ELSE AS AN ANSWER.\n"
    "5. Do not quote the expression.\n"
    "6. The data is stored in the `df` variable"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "This is the definition of all of the columns in the dataframe, there are no other columns besides this:\n"
    "{df_info_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)

response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

In [29]:
from llama_index.core.prompts import PromptTemplate

In [30]:
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5), df_info_str=df_info_str
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

### Custom Query Pipeline

We will use a custom query pipeline to preprocess the data before it is passed to the model. This will allow us to initially generate a query, run the query, and then synthesize a response based on the query result

In [31]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": Settings.llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": Settings.llm,
    },
    verbose=True,
)

qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])

qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)

qp.add_link("response_synthesis_prompt", "llm2")

### Querying

After running all of the following queries, you can view the traces in the Phoenix app on your browser to see exactly what is happening at each step.

In [39]:
response = qp.run(query_str="what is total cost of vet after 2022")
print(response)

> Running module input with input: 
query_str: what is total cost of vet after 2022

> Running module pandas_prompt with input: 
query_str: what is total cost of vet after 2022

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
     LOTID CREATIONDATE MODIFICATIONDATE  MISCOTHERCOST   FEEDCOST  \
0 ...

> Running module pandas_output_parser with input: 
input: assistant: (df['MODIFICATIONDATE'] > '2022-01-01').sum() * df['VETCOST'].mean()

> Running module response_synthesis_prompt with input: 
query_str: what is total cost of vet after 2022
pandas_instructions: assistant: (df['MODIFICATIONDATE'] > '2022-01-01').sum() * df['VETCOST'].mean()
pandas_output: 573988.1489118608

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: what is total cost of vet after 2022

Pandas Instructions (optional):
(